In [23]:
import os, sys, django, pandas as pd , numpy as np, requests, json, re, itertools, ftfy
from django.db.models import Q
from utils import TJData, setup_env, ElisAPI
# configura notebook para se utilizar recursos do django, se necessário
setup_env() 
from api import models as tj
# objeto para interagir com a minha API
api = ElisAPI('http://0.0.0.0:8000','elisdbadmin','elisdbpassword')

### Comarca
O bloco de código abaixo foi utilizado para analisar e decidir visualmente qual planilha referente a comarcas será utilizada, a planilha mapeada por `comarca` possui a coluna `COD_SERV` possui codigos distintos dos referenciados por demais planilhas
```
com_1 = TJData.read_csv(TJData.csv['comarca'],usecols=['COD_COM','NOME','NOME_REDU','COD_TJ'])
com_2 = TJData.read_csv(TJData.csv['comarca_2'], usecols=['COD_COMA','DESC_COMA','COD_TJ','DESC_REDU'])
com = pd.merge(left=com_1,right=com_2,how='inner',left_on='NOME_REDU', right_on='DESC_REDU')
```

#### Tratamento de dados

### Serventia

``` 
serv_1 = TJData.read_csv(TJData.csv['serventia_2'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'] )
serv_2 = TJData.read_csv(TJData.csv['serventias_full'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'])
serv = pd.merge(left=serv_1,right=serv_2,how='inner',left_on='COD_SERV', right_on='COD_SERV')
serv_outer = pd.merge(left=serv_1,right=serv_2,how='outer',left_on='COD_SERV', right_on='COD_SERV')
```
#### Tratamento de dados

### Competencias

### Tipo Personagem

### Assunto

### Classe

### Classe assunto

### Tipo movimento

### Tipo Andamento

### Cargo

### Funcionário

### Tipo Decisão Recurso

### Tipo Ato Juiz

### Ato Juiz

### Tipo Documento

### Processo

#### Obtem as serventias com função de Juizado Especial Cível das comarcas de Bangu e Campo Grande

In [2]:
serventias_jec = pd.DataFrame(
    list(map(lambda x: (x.comarca.desc_redu,x.comarca.desc_coma, x.cod_serv, x.desc_serv),
        tj.Serventia.objects.filter(
            Q(desc_serv__contains='Juizado Especial Cível'),
            Q(comarca__desc_redu='BAN') | Q(comarca__desc_redu='CGR')
    ))),
    columns=['comarca.desc_redu','comarca.desc_coma','cod_serv','desc_serv']
)
serventias_jec

,comarca.desc_redu,comarca.desc_coma,cod_serv,desc_serv
0,BAN,Regional de Bangu,204009,Cartório do 17º Juizado Especial Cível
1,BAN,Regional de Bangu,204509,17º Juizado Especial Cível
2,BAN,Regional de Bangu,204025,Cartório do 29º Juizado Especial Cível
3,BAN,Regional de Bangu,204525,29º Juizado Especial Cível
4,CGR,Regional de Campo Grande,205020,Cartório do 26º Juizado Especial Cível
5,CGR,Regional de Campo Grande,205520,26º Juizado Especial Cível
6,CGR,Regional de Campo Grande,205507,18º Juizado Especial Cível
7,CGR,Regional de Campo Grande,205007,Cartório do 18º Juizado Especial Cível


#### Obtem a lista de planilhas para processar resultantes codigos de serventias e do intervalo de anos selecinados 

####  carrega os processos que por algum motivo não tenham sido processados anteriormente

### Andamento Processo (Sentença)

In [3]:
servs_anos = TJData.build_serventias_anos_list(list(serventias_jec.cod_serv),range(2013,2017))
cod_procs = list(map(lambda x: x.cod_proc,tj.Processo.objects.all()))
juizes = list(map(lambda x: x.num_matr, tj.Funcionario.objects.all()))
tipos_andamento = list(map(lambda x: x.cod_tip_and, tj.TipoAndamento.objects.all()))
usecols = ['COD_PROC','ORDEM','COD_TIP_AND','NUM_MATR_JUIZ','COD_TIP_ATO','DT_ATO','COD_ATO','COD_SERV','COD_TIP_DEC_REC','TXT_DESCR','TXT_DESCR_RES']
mergecols = ['cod_proc','ordem']
files_andamentos_clob = TJData.list_files_in_serventias_anos(TJData.andamento_processo_clob,servs_anos)
files_andamentos = TJData.list_files_in_serventias_anos(TJData.andamento_processo,servs_anos)
files_andamentos += list(filter(lambda x : re.search(r'AndamentoProcesso_',x),files_andamentos_clob))
files_andamentos_clob = list(filter(lambda x : re.search(r'AndamentoProcessoClob_',x),files_andamentos_clob))

clean_andamentos_kwargs = {
    'usecols':usecols,
    'mergecols':mergecols,
    'files_clob_list':files_andamentos_clob,
    'clob_pattern':TJData.andamento_processo_clob,
    'juizes':juizes,
    'tipos_andamento':tipos_andamento,
}

In [4]:
# le em paralelo os arquivos referentes aos processos
andamento_dfs = TJData.concurrent_read(TJData.clean_andamentos, files_andamentos, clean_andamentos_kwargs)
andamento_dfs = pd.concat(andamento_dfs)

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [5]:
andamento_dfs = andamento_dfs.sort_values(['processo','ordem','juiz'])
andamento_dfs = andamento_dfs.drop_duplicates()
usecols = list(andamento_dfs.columns)
usecols.remove('num_seq')
andamento_dfs = andamento_dfs[usecols]
andamento_dfs = TJData.to_json(andamento_dfs)
andamento_dfs = api.split_list(andamento_dfs,150)

In [24]:
r_post = api.concurrent_request(api.post,'andamentosprocesso',andamento_dfs)
andamentos_post_erros = api.get_erros(r_post, 201)

In [36]:
erros = list(filter(lambda x: x.status_code != 201, map(lambda x: x[1] ,r_post)))
erros = list(map(lambda x: json.loads(x.request.body), erros))

In [37]:
retry_post = []
for erro in erros:
    retry_post.append(api.concurrent_request(api.post,'andamentosprocesso',erro))

In [44]:
list(map(lambda lst: list(map(lambda x: x[1].text, lst)),retry_post))

[['{"non_field_errors":["The fields processo, ordem, txt_descr_len must make a unique set."]}',
  '{"non_field_errors":["The fields processo, ordem, txt_descr_len must make a unique set."]}',
  '{"non_field_errors":["The fields processo, ordem, txt_descr_len must make a unique set."]}',
  '{"non_field_errors":["The fields processo, ordem, txt_descr_len must make a unique set."]}',
  '{"non_field_errors":["The fields processo, ordem, txt_descr_len must make a unique set."]}',
  '{"non_field_errors":["The fields processo, ordem, txt_descr_len must make a unique set."]}',
  '{"non_field_errors":["The fields processo, ordem, txt_descr_len must make a unique set."]}',
  '{"non_field_errors":["The fields processo, ordem, txt_descr_len must make a unique set."]}',
  '{"non_field_errors":["The fields processo, ordem, txt_descr_len must make a unique set."]}',
  '{"non_field_errors":["The fields processo, ordem, txt_descr_len must make a unique set."]}',
  '{"non_field_errors":["The fields proc

In [41]:
def list_text_response(lst):
    return

[[({'processo': '2013.204.000125-4',
    'ordem': '13',
    'tipo_andamento': '1',
    'juiz': '31941',
    'tipo_ato_juiz': '2',
    'dt_ato': '2014-12-15T00:00:00-02:00',
    'ato_juiz': '117',
    'serventia': '204009',
    'tipo_decisao_recurso': None,
    'txt_descr': 'Recebo os embargos de declaração, eis que tempestivos. No mérito,  dou-lhes provimento para revogar a tutela deferida à parte autora. No mais, mantenho a sentença como proferida. Intimem-se.',
    'txt_descr_len': 228},
   <Response [400]>),
  ({'processo': '2013.204.000547-8',
    'ordem': '10',
    'tipo_andamento': '1',
    'juiz': '31959',
    'tipo_ato_juiz': '2',
    'dt_ato': '2014-10-03T00:00:00-03:00',
    'ato_juiz': '8',
    'serventia': '204009',
    'tipo_decisao_recurso': None,
    'txt_descr': 'Dispensado o relatório na forma do artigo 38 da lei 9.099/95. Diante da manifestação da parte autora, homologo a desistência e, em consequência, JULGO EXTINTO O PROCESSO SEM RESOLUÇÃO DO MÉRITO, na forma do art